In [40]:
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np

data = pd.read_csv("/workspaces/bakery_sales_prediction/0_DataPreparation/00_data/data_long.csv")
#print(data.head())
#data brot:
df= data[data["Warengruppe_3"] == 1].copy()
data_wetter = pd.read_csv("/workspaces/bakery_sales_prediction/2_BaselineModel/02_Franz/weather_data_with_flags.csv")


df = df.drop(columns=["Warengruppe_1", "Warengruppe_2", "Warengruppe_3", "Warengruppe_4", "Warengruppe_5", "Warengruppe_6"])
df = df.drop(columns=["Datum"])
print(df)



          Umsatz  Temp_Very_Cold  Temp_Cold  Temp_Mild  Temp_Warm  Temp_Hot  \
3638  201.198426               0          0          1          0         0   
3639  265.261254               0          0          1          0         0   
3640  210.260241               0          0          0          1         0   
3641  190.686641               0          0          1          0         0   
3642  181.644870               0          0          1          0         0   
...          ...             ...        ...        ...        ...       ...   
5452  333.774099               0          0          0          0         1   
5453  371.819770               0          0          0          0         1   
5454  433.820697               0          0          0          1         0   
5455  348.770346               0          0          0          0         1   
5456  285.872616               0          0          0          0         1   

      Cloud_Clear  Cloud_Partly_Cloudy  Cloud_Cloud

In [41]:
# 1. Schritt 1: Untersuche lineare Korrelation zwischen den einzelnen Features mit der Zielvariable
# mit Korrelationsmatrix
import pandas as pd

# Korrelationsmatrix erstellen
correlation_matrix = df.corr()

# Korrelation mit der Zielvariable
correlation_with_target = correlation_matrix["Umsatz"].sort_values(ascending=False)
print(correlation_with_target)




Umsatz                    1.000000
Sommer                    0.632812
Monat_8                   0.467922
Monat_7                   0.430227
is_holiday                0.429896
Temp_Warm                 0.377216
wetter_sehr_schön         0.285412
Number_of_ships_scaled    0.269484
Number_of_ships           0.269484
Ship                      0.264904
Sonntag                   0.241471
Temp_Hot                  0.230605
Cloud_ok                  0.228773
Samstag                   0.206775
Temp_Mild                 0.184706
Cloud_Clear               0.147291
KielerWoche               0.126438
Cloud_Partly_Cloudy       0.118924
Feiertag                  0.116342
Ostertag                  0.096434
Wind_Moderate             0.079509
Monat_6                   0.076073
Heimspiel                 0.063103
VPI                       0.059597
Wind_ok                   0.043476
Monat_10                  0.022659
Weather_Good              0.021357
Silvester                 0.011746
Weather_Light_Issues

In [42]:
from sklearn.feature_selection import mutual_info_regression

# Nicht-lineare Zusammenhänge analysieren
X = df.drop("Umsatz", axis=1)
y = df["Umsatz"]
mi = mutual_info_regression(X, y)
mi_series = pd.Series(mi, index=X.columns).sort_values(ascending=False)
print(mi_series)


VPI                       0.258842
Sommer                    0.224970
Temp_Cold                 0.156736
Winter                    0.128611
Monat_8                   0.103448
Werktag                   0.093452
Monat_7                   0.091562
Temp_Warm                 0.089850
is_holiday                0.088034
Temp_Mild                 0.069399
Sonntag                   0.062396
Weihnachtsmarkt           0.044659
wetter_sehr_schön         0.038849
Number_of_ships_scaled    0.037599
Number_of_ships           0.037535
Monat_1                   0.034566
Monat_12                  0.027470
Herbst                    0.026533
Cloud_Cloudy              0.025793
Cloud_ok                  0.025762
Monat_9                   0.025499
Ship                      0.025205
Cloud_Clear               0.024482
Samstag                   0.024440
Monat_3                   0.024324
Monat_10                  0.023072
Temp_Hot                  0.022956
Monat_2                   0.022818
Monat_6             

In [43]:
# Features mit geringer linearer und nichtlinearer Korrelation finden
low_linear_corr_features = correlation_with_target[correlation_with_target.abs() < 0.1].index
low_nonlinear_corr_features = mi_series[mi_series < 0.02].index

# Schnittmenge der Features mit beiden Bedingungen
low_corr_features = set(low_linear_corr_features).intersection(low_nonlinear_corr_features)

# Ausgabe der Features
#print("Features mit geringer linearer und nichtlinearer Korrelation:", list(low_corr_features))

# DataFrame erstellen für die Ausgabe in Tabellenform
low_corr_features_table = pd.DataFrame({
    "Feature": list(low_corr_features),
    "Lineare Korrelation": [correlation_with_target[feature] for feature in low_corr_features],
    "Nichtlineare Korrelation": [mi_series[feature] for feature in low_corr_features]
})


# Tabelle nach der linearen Korrelation sortieren
low_corr_features_table = low_corr_features_table.sort_values(by="Lineare Korrelation", ascending = False)

# Ausgabe der sortierten Tabelle
print(low_corr_features_table)



                 Feature  Lineare Korrelation  Nichtlineare Korrelation
10              Ostertag             0.096434                  0.007079
14         Wind_Moderate             0.079509                  0.019668
11             Heimspiel             0.063103                  0.000889
8                Wind_ok             0.043476                  0.000000
9           Weather_Good             0.021357                  0.000000
6              Silvester             0.011746                  0.002454
3   Weather_Light_Issues             0.007030                  0.000000
16               Monat_5             0.004995                  0.004636
15   zwischen_den_jahren             0.001746                  0.008471
5         Weather_Severe            -0.007125                  0.002826
0                  Markt            -0.008997                  0.016002
12      Weather_Moderate            -0.023788                  0.000917
2            Wind_Strong            -0.043476                  0

In [44]:

import numpy as np

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Stark korrelierte Features (> 0.5) finden
high_corr = corr_matrix.abs() > 0.7
np.fill_diagonal(high_corr.values, False)  # Hauptdiagonale ignorieren
correlated_features = [column for column in high_corr.columns if any(high_corr[column])]
print(f"Stark korrelierte Features:", correlated_features)

Stark korrelierte Features: ['Cloud_Cloudy', 'Wind_Light', 'Wind_Moderate', 'Wind_Strong', 'Number_of_ships', 'Ship', 'Weihnachtsmarkt', 'Monat_12', 'Number_of_ships_scaled', 'Cloud_ok', 'Wind_ok']


In [45]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# VIF für alle Features berechnen
X = df.drop("Umsatz", axis=1)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)



/workspaces/bakery_sales_prediction/.venv/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                   Feature           VIF
0           Temp_Very_Cold  1.169477e+08
1                Temp_Cold           inf
2                Temp_Mild           inf
3                Temp_Warm           inf
4                 Temp_Hot  1.500577e+07
5              Cloud_Clear  8.145413e+05
6      Cloud_Partly_Cloudy  1.169889e+06
7             Cloud_Cloudy  1.091062e+05
8               Wind_Light  3.870857e+04
9            Wind_Moderate  1.168463e+05
10             Wind_Strong  2.113227e+06
11            Weather_Good  1.182944e+07
12    Weather_Light_Issues  8.581310e+04
13        Weather_Moderate  3.167107e+05
14          Weather_Severe  1.810797e+07
15             KielerWoche  1.453789e+00
16                  Montag  1.906687e+06
17                Dienstag  2.154440e+06
18                Mittwoch  5.825325e+05
19              Donnerstag  1.072312e+05
20                 Freitag  6.563315e+05
21                 Samstag  1.192699e+05
22                 Sonntag  2.503423e+06
23              

In [46]:
# suche miteinander korrelierrende features


In [47]:

# Korrelationsmatrix berechnen
corr_matrix = df.corr()

# Schwellenwert für "stark korreliert"
threshold = 0.4

# Finde die Paare mit starker Korrelation
strong_corr_pairs = set()

# Iteriere über die obere Dreiecksmatrix der Korrelationsmatrix
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:  # Korrelation über dem Schwellenwert
            strong_corr_pairs.add((corr_matrix.columns[i], corr_matrix.columns[j]))

# Anzeigen der Paare starker Korrelationen
print("Stark korrelierte Feature-Paare:")
print(strong_corr_pairs)


Stark korrelierte Feature-Paare:
{('Number_of_ships_scaled', 'Ship'), ('Wind_ok', 'Wind_Moderate'), ('Cloud_ok', 'Cloud_Cloudy'), ('Werktag', 'Sonntag'), ('Werktag', 'Samstag'), ('Temp_Mild', 'Temp_Cold'), ('Monat_6', 'KielerWoche'), ('Cloud_ok', 'Cloud_Clear'), ('Monat_8', 'Umsatz'), ('Monat_11', 'Herbst'), ('Winter', 'Temp_Mild'), ('Sommer', 'Temp_Warm'), ('Cloud_Cloudy', 'Cloud_Clear'), ('Monat_6', 'Sommer'), ('zwischen_den_jahren', 'Silvester'), ('Monat_7', 'Sommer'), ('Monat_1', 'Winter'), ('wetter_sehr_schön', 'Temp_Hot'), ('Ship', 'Number_of_ships'), ('Weather_Light_Issues', 'Weather_Good'), ('Monat_4', 'Frühling'), ('Monat_12', 'Weihnachtsmarkt'), ('Weather_Moderate', 'Weather_Light_Issues'), ('Sommer', 'Umsatz'), ('Winter', 'Umsatz'), ('Monat_4', 'Markt'), ('Temp_Cold', 'Umsatz'), ('Monat_7', 'Umsatz'), ('Cloud_ok', 'Cloud_Partly_Cloudy'), ('Monat_2', 'Winter'), ('Monat_3', 'Frühling'), ('Number_of_ships_scaled', 'Number_of_ships'), ('Wind_Moderate', 'Wind_Light'), ('Sommer', 